In [1]:

import cv2 as cv

import glob
import numpy as np

In [2]:

%matplotlib tk
import matplotlib.pyplot as plt
import numpy as np


from PIL import Image

In [4]:
left_image_path = 'data/7_left.JPG'
right_image_path = 'data/7_right.JPG'


im_left = cv.imread(left_image_path, 1)

im_right = cv.imread(right_image_path, 1)
# plt.figure()
# plt.imshow(im_left) 
# plt.show()

In [5]:
# idea here we need a chessboard to calibrate the camera, world_scaling can be the real size 
rows = 6
cols = 7
world_scaling = 1

In [6]:
def find_carmera_matrix(image_path):
    image = cv.imread(image_path)
    objp = np.zeros((rows*cols,3), np.float32)
    objp[:,:2] = np.mgrid[0:rows,0:cols].T.reshape(-1,2)
    objp = objp * world_scaling
    width = image.shape[1]
    height = image.shape[0]   

    img_point = []
    points_3d = []
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    ret, corners = cv.findChessboardCorners(gray, (rows,cols), cv.CALIB_CB_NORMALIZE_IMAGE)
    if ret == True:
        corners2 = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        img_point.append(corners2)
        points_3d.append(objp)
        cv.drawChessboardCorners(image, (rows,cols), corners2, ret)
        cv.imshow('image', image)
        cv.waitKey(0)
        cv.destroyAllWindows()
    else:
        print('corners not found')
    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(points_3d,img_point, (width, height), None, None)
    print(ret)
    return mtx, dist, img_point, points_3d



In [7]:
mtx_R, dist_R, img_point_R, points_3d_R = find_carmera_matrix(right_image_path)

mtx_L, dist_L, img_point_L, points_3d_L = find_carmera_matrix(left_image_path)


 
# print(points_3d_R)
# print(points_3d_L)0

2024-12-18 22:55:43.939 python[74382:9056941] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-18 22:55:43.939 python[74382:9056941] +[IMKInputSession subclass]: chose IMKInputSession_Modern


0.15480580288616166
0.10368479668552631


In [8]:
print(dist_L)
print(dist_R)


width = im_left.shape[1]
height = im_left.shape[0]
print(width, height)
width = im_right.shape[1]
height = im_right.shape[0]
print(width, height)

[[ 4.67477052e-01 -3.66964417e+00  6.38064550e-03 -1.78085008e-02
   1.03751327e+01]]
[[ 0.10413267 -0.56309796  0.02129309  0.00703079 -4.49056238]]
5712 4284
5712 4284


In [9]:
def find_stereo_camera_matrix(mtx_L, dist_L, mtx_R, dist_R, img_point_L, img_point_R, points_3d):
    stereocalibration_flags = cv.CALIB_FIX_INTRINSIC
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 100, 0.0001)
    ret, CM1, dist1, CM2, dist2, R, T, E, F = cv.stereoCalibrate(points_3d, img_point_L, img_point_R, mtx_L, dist_L,
                                mtx_R, dist_R, (width, height), criteria = criteria, flags = stereocalibration_flags)
    return R, T


In [10]:
R,T = find_stereo_camera_matrix(mtx_L, dist_L, mtx_R, dist_R, img_point_L, img_point_R, points_3d_L)

In [11]:
RT1 = np.concatenate([np.eye(3), [[0],[0],[0]]], axis = -1)
P1 = mtx_L @ RT1

RT2 = np.concatenate([R, T], axis = -1)
P2 = mtx_R @ RT2


In [12]:
def find_center(M, shown = False):
    U, S, V = np.linalg.svd(M)
    if shown:
        print(S)
        print("Camera Center:", V[-1, :3] / V[-1, -1])
    return V[-1, :3] / V[-1, -1]

In [13]:
def DLT(P1, P2, point1, point2):
 
    A = [point1[1]*P1[2,:] - P1[1,:],
         P1[0,:] - point1[0]*P1[2,:],
         point2[1]*P2[2,:] - P2[1,:],
         P2[0,:] - point2[0]*P2[2,:]
        ]
    A = np.array(A).reshape((4,4))
    #print('A: ')
    #print(A)
 
    B = A.transpose() @ A
    from scipy import linalg
    U, s, Vh = linalg.svd(B, full_matrices = False)
 
    # print('Triangulated point: ')
    # print(Vh[3,0:3]/Vh[3,3])
    return Vh[3,0:3]/Vh[3,3]

In [14]:
pt = DLT(P1, P2, img_point_L[0][0][0], img_point_R[0][0][0])
print(pt)

[20.38241371  3.32959346 69.38126954]


In [15]:
pt = DLT(P1, P2, img_point_L[0][1][0], img_point_R[0][1][0])

In [16]:
c1 = find_center(P1)
c2 = find_center(P2)

print(c1)
print(c2)


[0. 0. 0.]
[35.63069593 -7.357921   23.75061013]


In [17]:
def get_input_lines(im, min_lines=3):
    n = 0

    plt.figure()
    plt.imshow(im)
    plt.show()
    print('Set 3 points to give the plane of table')

    clicked = plt.ginput(3, timeout=0, show_clicks=True)
    if not clicked or len(clicked) < 3:
        print('Need at least %d lines, you have %d now' % (min_lines, n))


    
    pt1 = np.array([clicked[0][0], clicked[0][1], 1])
    pt2 = np.array([clicked[1][0], clicked[1][1], 1])
    pt3 = np.array([clicked[2][0], clicked[2][1], 1])

    return [pt1, pt2, pt3]

In [18]:
pts_left = (get_input_lines(im_left, 3))
pts_right = (get_input_lines(im_right, 3))

Set 3 points to give the plane of table
Set 3 points to give the plane of table


In [19]:
pts_x = []
pts_y = []
pts_z = []

for i in range(img_point_L[0].shape[0]):
    pt = DLT(P1, P2, img_point_L[0][i][0], img_point_R[0][i][0])
    # pt = pt / pt[2]
    pts_x.append(pt[0])
    pts_y.append(pt[1])
    pts_z.append(pt[2])
print(len(pts_x), len(pts_y), len(pts_z))


42 42 42


In [20]:
pt_table = np.array([DLT(P1, P2, pts_left[0], pts_right[0]), DLT(P1, P2, pts_left[1], pts_right[1]), DLT(P1, P2, pts_left[2], pts_right[2])])

v1 = pt_table[0] - pt_table[1]
v2 = pt_table[0] - pt_table[2]
normal = np.cross(v1, v2) / np.linalg.norm(np.cross(v1, v2))

d = -np.dot(normal, pt_table[0])



In [21]:
fig = plt.figure()

ax = fig.add_subplot(111, projection='3d')

pts_x_c = pts_x + [c1[0], c2[0]]
pts_y_c = pts_y + [c1[1], c2[1]]
pts_z_c = pts_z + [c1[2], c2[2]]


max_range = max([max(pts_x_c)-min(pts_x_c), max(pts_y_c)-min(pts_y_c), max(pts_z_c)-min(pts_z_c)])
mid_x = (max(pts_x_c) + min(pts_x_c)) * 0.5
mid_y = (max(pts_y_c) + min(pts_y_c)) * 0.5
mid_z = (max(pts_z_c) + min(pts_z_c)) * 0.5
ax.view_init(elev=135, azim=0)
ax.set_xlim(mid_x - max_range/2, mid_x + max_range/2)
ax.set_ylim(mid_y - max_range/2, mid_y + max_range/2)
ax.set_zlim(mid_z - max_range/2, mid_z + max_range/2)

ax.scatter(pts_x, pts_y, pts_z, c='b')  
ax.scatter(c1[0], c1[1], c1[2], c='r')
ax.scatter(c2[0], c2[1], c2[2], c='r')
ax.scatter(pt_table[0][0], pt_table[0][1], pt_table[0][2], c='g')
ax.scatter(pt_table[1][0], pt_table[1][1], pt_table[1][2], c='g')
ax.scatter(pt_table[2][0], pt_table[2][1], pt_table[2][2], c='g')
X_range = np.linspace(mid_x - max_range/2, mid_x + max_range/2, 10)
Y_range = np.linspace(mid_y - max_range/2, mid_y + max_range/2, 10)
ax.legend(['Chessboard', 'Camera 1', 'Camera 2', 'Table'])
n_x, n_y, n_z = normal
X, Y = np.meshgrid(X_range, Y_range)
Z = (-d - n_x * X - n_y * Y) / n_z
ax.plot_surface(X, Y, Z, alpha=0.5, color='cyan')
plt.show()

In [ ]:

def find_plane_two_vectors(c, d1, d2):

    n = np.cross(d1, d2)  
    if np.linalg.norm(n) == 0:
        raise ValueError("Direction vectors are collinear; no unique plane can be defined.")
    
    n = n / np.linalg.norm(n) 

    d = -np.dot(n, c) 

    return n, d

In [ ]:

def backproject(P, x):


    x_h = np.array([x[0], x[1], 1])  
    M = P[:, :3]  
    d = np.linalg.inv(M) @ x_h  
    d = d / np.linalg.norm(d) 
    return d



In [107]:
def intersection_line_of_planes(n1, d1, n2, d2):

    d =  np.cross(n1, n2)

    if np.linalg.norm(d) == 0:
        raise ValueError("Planes are parallel, no intersection line.")
    

    A = np.vstack((n1, n2, d)) 
    b = np.array([d1, d2, 0])  
    
    # Solve the linear system for X0
    X0 = - np.linalg.lstsq(A, b, rcond=None)[0]
    
    return X0, d


In [25]:
import sys
import os
import matplotlib.pyplot as plt
sys.path.append(os.path.abspath('../utils'))
sys.path.append(os.path.abspath('../'))
import model2 as model2_module
import image as image_module
print(os.getcwd())
os.chdir('../')

model2_module.load_model()

/Users/kangyufeng/Documents/Repo/Iye/stereoCalibrate
load gaze estimator
load the pre-trained model:  ckpt/epoch_24_ckpt.pth.tar


In [26]:
image_path = "stereoCalibrate/data/7_left.JPG"  
img, small = image_module.load_img(image_path)



image_module.set_img(small)

print(img.shape)
# gaze estimation

arrow_start_left, arrow_end_left = model2_module.find_vector_arrow(img)


(4284, 5712, 3)
detected_faces:  3
pred_gaze_np:  [-0.520068    0.13888443]


In [27]:
plt.figure()
plt.imshow(img)
plt.scatter(arrow_start_left[0], arrow_start_left[1], c='r')
plt.show()



In [28]:
image_path = "stereoCalibrate/data/7_right.JPG"  
img, small = image_module.load_img(image_path)
# image_module.set_img(small)

arrow_start_right, arrow_end_right = model2_module.find_vector_arrow(img)

detected_faces:  3
pred_gaze_np:  [-0.48817384  0.65958476]


In [29]:
plt.figure()
plt.imshow(img)
plt.scatter(arrow_start_right[0], arrow_start_right[1], c='r')
plt.show()



In [30]:
face_3d = DLT(P1, P2, arrow_start_left, arrow_start_right)



In [61]:
v_left_1 = backproject(P1, arrow_start_left)
v_left_2 = backproject(P1, arrow_end_left)

v_right_1 = backproject(P2, arrow_start_right)
v_right_2 = backproject(P2, arrow_end_right)

n_left, d_left = find_plane_two_vectors(face_3d, v_left_1, v_left_2)
n_right, d_right = find_plane_two_vectors(c2, v_right_1, v_right_2)

x, d_line = intersection_line_of_planes(n_left, d_left, n_right, d_right)





In [80]:
fig = plt.figure()

ax = fig.add_subplot(111, projection='3d')

pts_x_c = pts_x + [c1[0], c2[0]]
pts_y_c = pts_y + [c1[1], c2[1]]
pts_z_c = pts_z + [c1[2], c2[2]]


max_range = max([max(pts_x_c)-min(pts_x_c), max(pts_y_c)-min(pts_y_c), max(pts_z_c)-min(pts_z_c)])
mid_x = (max(pts_x_c) + min(pts_x_c)) * 0.5
mid_y = (max(pts_y_c) + min(pts_y_c)) * 0.5
mid_z = (max(pts_z_c) + min(pts_z_c)) * 0.5
ax.view_init(elev=135, azim=0)
ax.set_xlim(mid_x - max_range/2, mid_x + max_range/2)
ax.set_ylim(mid_y - max_range/2, mid_y + max_range/2)
ax.set_zlim(mid_z - max_range/2, mid_z + max_range/2)

ax.scatter(pts_x, pts_y, pts_z, c='b', label='Chessboard')
ax.scatter(c1[0], c1[1], c1[2], c='r', label='Camera 1')
ax.scatter(c2[0], c2[1], c2[2], c='purple',  label='Camera 2')
ax.scatter(face_3d[0], face_3d[1], face_3d[2], c='black', label='Face')
ax.scatter(pt_table[0][0], pt_table[0][1], pt_table[0][2], c='g', label='Table')
ax.scatter(pt_table[1][0], pt_table[1][1], pt_table[1][2], c='g' )
ax.scatter(pt_table[2][0], pt_table[2][1], pt_table[2][2], c='g')
X_range = np.linspace(mid_x - max_range/2, mid_x + max_range/2, 10)
Y_range = np.linspace(mid_y - max_range/2, mid_y + max_range/2, 10)
n_x, n_y, n_z = normal
X, Y = np.meshgrid(X_range, Y_range)
Z = (-d - n_x * X - n_y * Y) / n_z
ax.plot_surface(X, Y, Z, alpha=0.5, color='cyan')
t = np.linspace(-200, 200, 100)

# X_line = c2[0] + t * v_right_1[0]
# Y_line = c2[1] + t * v_right_1[1]
# Z_line = c2[2] + t * v_right_1[2]
# ax.plot(X_line, Y_line, Z_line, c='r')
# ax.plot_surface(X, Y, Z, alpha=0.5, color='cyan')

# X = c2[0] + t * v_right_2[0]    
# Y = c2[1] + t * v_right_2[1]
# Z = c2[2] + t * v_right_2[2]
# ax.plot(X, Y, Z, c='r')
# Z2 = (-d_left - n_left[0] * X - n_left[1] * Y) / n_left[2]
# Z3 = (-d_right - n_right[0] * X - n_right[1] * Y) / n_right[2]

# ax.plot_surface(X, Y, Z2, alpha=0.5, color='yellow')
# ax.plot_surface(X, Y, Z3, alpha=0.5, color='red')

X = x[0] + t * d_line[0]
Y = x[1] + t * d_line[1]
Z = x[2] + t * d_line[2]


ax.plot(X, Y, Z, c='orange', label='Gaze vector')
ax.legend()
plt.show()

In [37]:
os.getcwd()

'/Users/kangyufeng/Documents/Repo/Iye'

In [53]:
import sys
import os

sys.path.append(os.path.abspath('../utils'))
sys.path.append(os.path.abspath('../'))


import model1 as model_module
model_module.time_show()

model_module.load_model("vit_b", "./vit_b.pth", "sam")

Time will not be shown
Reloading model
Using SAM
Model loaded


In [54]:
import image as image_module
import mask as mask_module
img, small = image_module.load_img("stereoCalibrate/data/7_left.JPG")



image_module.set_img(small)
masks = model_module.mask_generate(small)

masks_on_table = mask_module.find_object_on_table(masks)

pt_mask_left = []
for mask in masks_on_table:
    y_coords, x_coords = np.where(mask["segmentation"])
    x = np.mean(x_coords)
    y = np.mean(y_coords)
    pt_mask_left.append([x, y])


img, small = image_module.load_img("stereoCalibrate/data/7_right.JPG")
image_module.set_img(small)
masks = model_module.mask_generate(small)

masks_on_table = mask_module.find_object_on_table(masks)

pt_mask_right = []
for mask in masks_on_table:
    y_coords, x_coords = np.where(mask["segmentation"])
    x = np.mean(x_coords)
    y = np.mean(y_coords)
    pt_mask_right.append([x, y])
    



Generating mask
Generating mask


In [55]:
plt.figure()
plt.imshow(small)
for pt in pt_mask_right:
    plt.scatter(pt[0], pt[1], c='r')
plt.show()


In [56]:
pt_mask_left = np.array(pt_mask_left) * 10
pt_mask_right = np.array(pt_mask_right) * 10

In [57]:


pt_mask_3d = []
for i in range(len(pt_mask_left)):
    min_dis = 1000000
    cur_pt = None
    for j in range(len(pt_mask_right)):
        pt = DLT(P1, P2, pt_mask_left[i], pt_mask_right[j])
        dist = abs(np.dot(normal, pt ) + d)
        if dist < min_dis:
            min_dis = dist
            cur_pt = pt
        # pt_mask_3d.append(pt)
    pt_mask_3d.append(cur_pt)

In [58]:
print(pt_mask_3d)

[array([23.7830731 ,  1.19315716, 96.28395367]), array([-11.5422322 ,  15.05381154,  66.8557376 ]), array([50.07130345,  6.93747106, 96.34528966]), array([-36.16132434,  15.72617161,  70.36047831]), array([ -3.91165204,  -3.65846285, 102.45926751]), array([26.06143376,  6.89472649, 87.80712468]), array([28.22869526,  9.19142849, 81.32617438]), array([-27.36887032,  12.29997642,  64.55775227]), array([22.81523072,  1.3719749 , 98.69124631]), array([-11.49045032,  13.77491148,  66.53401076]), array([-11.45055615,  12.56454748,  66.23688068]), array([29.02890472,  3.91551389, 94.12351068]), array([2.10262289e+01, 4.80674398e-02, 1.06489896e+02]), array([27.15914325,  8.21818262, 85.96563043]), array([24.65792341,  2.7803859 , 93.35534963]), array([-38.90990588,  17.4528628 ,  72.27740461]), array([ 48.35919175, -10.66940787, 109.52309204]), array([-11.52814267,  13.79347007,  66.5685962 ]), array([32.37541549,  7.98082448, 74.58819295]), array([43.91805571, -8.16284107, 77.74579663]), arr

In [59]:
pt_mask_3d

[array([23.7830731 ,  1.19315716, 96.28395367]),
 array([-11.5422322 ,  15.05381154,  66.8557376 ]),
 array([50.07130345,  6.93747106, 96.34528966]),
 array([-36.16132434,  15.72617161,  70.36047831]),
 array([ -3.91165204,  -3.65846285, 102.45926751]),
 array([26.06143376,  6.89472649, 87.80712468]),
 array([28.22869526,  9.19142849, 81.32617438]),
 array([-27.36887032,  12.29997642,  64.55775227]),
 array([22.81523072,  1.3719749 , 98.69124631]),
 array([-11.49045032,  13.77491148,  66.53401076]),
 array([-11.45055615,  12.56454748,  66.23688068]),
 array([29.02890472,  3.91551389, 94.12351068]),
 array([2.10262289e+01, 4.80674398e-02, 1.06489896e+02]),
 array([27.15914325,  8.21818262, 85.96563043]),
 array([24.65792341,  2.7803859 , 93.35534963]),
 array([-38.90990588,  17.4528628 ,  72.27740461]),
 array([ 48.35919175, -10.66940787, 109.52309204]),
 array([-11.52814267,  13.79347007,  66.5685962 ]),
 array([32.37541549,  7.98082448, 74.58819295]),
 array([43.91805571, -8.16284107,

In [ ]:
fig = plt.figure()

ax = fig.add_subplot(111, projection='3d')

pts_x_c = pts_x + [c1[0], c2[0]]
pts_y_c = pts_y + [c1[1], c2[1]]
pts_z_c = pts_z + [c1[2], c2[2]]


max_range = max([max(pts_x_c)-min(pts_x_c), max(pts_y_c)-min(pts_y_c), max(pts_z_c)-min(pts_z_c)])
mid_x = (max(pts_x_c) + min(pts_x_c)) * 0.5
mid_y = (max(pts_y_c) + min(pts_y_c)) * 0.5
mid_z = (max(pts_z_c) + min(pts_z_c)) * 0.5
ax.view_init(elev=135, azim=0)
ax.set_xlim(mid_x - max_range/2, mid_x + max_range/2)
ax.set_ylim(mid_y - max_range/2, mid_y + max_range/2)
ax.set_zlim(mid_z - max_range/2, mid_z + max_range/2)

ax.scatter(pts_x, pts_y, pts_z, c='b', label='Chessboard')
ax.scatter(c1[0], c1[1], c1[2], c='r', label='Camera 1')
ax.scatter(c2[0], c2[1], c2[2], c='purple',  label='Camera 2')
ax.scatter(face_3d[0], face_3d[1], face_3d[2], c='black', label='Face')
ax.scatter(pt_table[0][0], pt_table[0][1], pt_table[0][2], c='g', label='Table')
ax.scatter(pt_table[1][0], pt_table[1][1], pt_table[1][2], c='g' )
ax.scatter(pt_table[2][0], pt_table[2][1], pt_table[2][2], c='g')
X_range = np.linspace(mid_x - max_range/2, mid_x + max_range/2, 10)
Y_range = np.linspace(mid_y - max_range/2, mid_y + max_range/2, 10)
n_x, n_y, n_z = normal
X, Y = np.meshgrid(X_range, Y_range)
Z = (-d - n_x * X - n_y * Y) / n_z
ax.plot_surface(X, Y, Z, alpha=0.5, color='cyan')
t = np.linspace(-200, 200, 100)

# X_line = c2[0] + t * v_right_1[0]
# Y_line = c2[1] + t * v_right_1[1]
# Z_line = c2[2] + t * v_right_1[2]
# ax.plot(X_line, Y_line, Z_line, c='r')
# ax.plot_surface(X, Y, Z, alpha=0.5, color='cyan')

# X = c2[0] + t * v_right_2[0]    
# Y = c2[1] + t * v_right_2[1]
# Z = c2[2] + t * v_right_2[2]
# ax.plot(X, Y, Z, c='r')
# Z2 = (-d_left - n_left[0] * X - n_left[1] * Y) / n_left[2]
# Z3 = (-d_right - n_right[0] * X - n_right[1] * Y) / n_right[2]

# ax.plot_surface(X, Y, Z2, alpha=0.5, color='yellow')
# ax.plot_surface(X, Y, Z3, alpha=0.5, color='red')

X = x[0] + t * d_line[0]
Y = x[1] + t * d_line[1]
Z = x[2] + t * d_line[2]


x_vals = [pt[0] for pt in pt_mask_3d]
y_vals = [pt[1] for pt in pt_mask_3d]
z_vals = [pt[2] for pt in pt_mask_3d]

ax.scatter(x_vals, y_vals, z_vals, c='orange', label='object')
# ax.scatter(pt_insterect[0], pt_insterect[1], pt_insterect[2], c='grey', label='object')
# ax.scatter(x_vals, y_vals, z_vals, c='orange', label='object')
ax.legend()


ax.plot(X, Y, Z, c='orange', label='Gaze vector')
plt.show()

In [ ]:
def find_closest_point(points, x0, d):

    
    closest_point = None
    index = -1
    min_dist_sq = float('inf')
    
    dd = d[0]*d[0] + d[1]*d[1] + d[2]*d[2]  # d·d
    
    for i in range(len(points)):
        p = points[i]
        px, py, pz = p
        t = ((px - x0[0])*d[0] + (py - x0[1])*d[1] + (pz - x0[2])*d[2]) / dd

        proj = [x0[0] + t*d[0], x0[1] + t*d[1], x0[2] + t*d[2]]
        
        dx = p[0] - proj[0]
        dy = p[1] - proj[1]
        dz = p[2] - proj[2]
        dist_sq = dx*dx + dy*dy + dz*dz

        if dist_sq < min_dist_sq:
            min_dist_sq = dist_sq
            closest_point = p
            index = i
    
    return closest_point, index, min_dist_sq

In [ ]:
closest_point, index = find_closest_point(pt_mask_3d, x, d_line)
pt_mask_3d.pop(index)
second_closest_point, index2 = find_closest_point(pt_mask_3d, x, d_line) 
pt


In [79]:
image_module.set_img(small)
image_module.show_box( masks_on_table[index]["bbox"])

In [69]:
img, small = image_module.load_img("stereoCalibrate/data/7_left.JPG")



image_module.set_img(small)
masks = model_module.mask_generate(small)

masks_on_table = mask_module.find_object_on_table(masks)

Generating mask


In [78]:
pt_mask_3d.pop(index)
closest_point, index = find_closest_point(pt_mask_3d, x, d_line)


In [ ]:
def line_plane_intersection(x0, d, n, c):

    # Compute denominators and numerators
    ndotd = n[0]*d[0] + n[1]*d[1] + n[2]*d[2]
    if ndotd == 0:

        ndotx0 = n[0]*x0[0] + n[1]*x0[1] + n[2]*x0[2]
        # if abs(ndotx0 - c) < 1e-9:

        #     return None  # or return x0 indicating one point on it
        # else:
        #     # No intersection
        #     return None

    ndotx0 = n[0]*x0[0] + n[1]*x0[1] + n[2]*x0[2]
    t = (c - ndotx0) / ndotd

    xi = x0[0] + t*d[0]
    yi = x0[1] + t*d[1]
    zi = x0[2] + t*d[2]

    return [xi, yi, zi]



In [105]:
pt_insterect = line_plane_intersection(x, d_line, normal, -d)

In [106]:
pt_insterect = np.append(pt_insterect, 1)
print(pt_insterect)
pt_insterect = np.dot(P2, pt_insterect)
pt_insterect = pt_insterect / pt_insterect[2]

plt.figure()
im_right_bg = cv.cvtColor(im_right, cv.COLOR_BGR2RGB)
plt.imshow(im_right_bg)
plt.scatter(pt_insterect[0], pt_insterect[1], c='r')
plt.show()

[-15.1019667   22.87715318  44.22813513   1.        ]
